## Key Insights
1. Highly safe model
2. Outperforms similar size model in 11 out of 18 tasks
3. Follows the decoder architecture in the Attention is All you Need paper with some modification of components based on the recent advancements
4. The changes in the architecture are MQA, RMSNorm on both input and output and GeGLU instead of ReLU
5. Better performance in terms of safety and accuracy across benchmarks over Mistral

In [ ]:
! pip install trl peft accelerate datasets bitsandbytes git+https://github.com/huggingface/transformers

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# Instruction Tuning - SFT

In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
from accelerate import Accelerator
import os

In [ ]:
sft_config = {
            "model_ckpt": "google/gemma-2b-it",
            "load_in_4bit": True,
            "device_map": {"": Accelerator().local_process_index},
            "torch_dtype": torch.float16,
            "trust_remote_code": True,
            "use_lora": True,
            "r": 16,
            "lora_alpha": 16,
            "lora_dropout": 0.05,
            "bias": "none",
            "task_type": "CAUSAL_LM",
            "target_modules": ["q_proj", "v_proj"],
            "output_dir": "gemma-chatbot",
            "per_device_train_batch_size": 1,
            "gradient_accumulation_steps": 1,
            "optim": "paged_adamw_32bit",
            "learning_rate": 2e-4,
            "lr_scheduler_type": "cosine",
            "save_strategy": "epoch",
            "logging_steps": 100,
            "num_train_epochs": 1,
            "max_steps": 250,
            "fp16": True,
            "push_to_hub": True,
            "train_cln_name": "text",
            "packing": False,
            "max_seq_length": 512,
            "neftune_noise_alpha": 5
        }

In [ ]:
class TrainSFT:

    def __init__(self, data, config):
        self.data = data
        self.config = config

    def prepare_lora_model(self):

        self.lora_config = LoraConfig(
                                    r=self.config["r"],
                                    lora_alpha=self.config["lora_alpha"],
                                    lora_dropout=self.config["lora_dropout"],
                                    bias=self.config["bias"],
                                    task_type=self.config["task_type"],
                                    target_modules=self.config["target_modules"]
                                )
        self.model = get_peft_model(self.model, self.lora_config)
    def load_model_tokenizer(self):

        self.model = AutoModelForCausalLM.from_pretrained(
                            self.config["model_ckpt"],
                            load_in_4bit=self.config["load_in_4bit"],
                            device_map=self.config["device_map"],
                            torch_dtype=self.config["torch_dtype"]
                        )
        self.model.config.use_cache=False
        self.model.config.pretraining_tp=1
        self.model = prepare_model_for_kbit_training(self.model)
        if self.config["use_lora"]:
            self.prepare_lora_model()

        self.tokenizer = AutoTokenizer.from_pretrained(self.config["model_ckpt"])
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def set_training_args(self):

        return TrainingArguments(
                                    output_dir=self.config["output_dir"],
                                    per_device_train_batch_size=self.config["per_device_train_batch_size"],
                                    gradient_accumulation_steps=self.config["gradient_accumulation_steps"],
                                    optim=self.config["optim"],
                                    learning_rate=self.config["learning_rate"],
                                    lr_scheduler_type=self.config["lr_scheduler_type"],
                                    save_strategy=self.config["save_strategy"],
                                    logging_steps=self.config["logging_steps"],
                                    num_train_epochs=self.config["num_train_epochs"],
                                    max_steps=self.config["max_steps"],
                                    fp16=self.config["fp16"],
                                    push_to_hub=self.config["push_to_hub"]
                                )

    def create_trainer(self):

        self.load_model_tokenizer()
        if self.config["use_lora"]:
            print(self.model.print_trainable_parameters())
            self.trainer = SFTTrainer(
                                    model=self.model,
                                    train_dataset=self.data,
                                    peft_config=self.lora_config,
                                    dataset_text_field=self.config["train_cln_name"],
                                    args=self.set_training_args(),
                                    tokenizer=self.tokenizer,
                                    packing=self.config["packing"],
                                    max_seq_length=self.config["max_seq_length"]
                                )
        else:
            self.trainer = SFTTrainer(
                                    model=self.model,
                                    train_dataset=self.data,
                                    dataset_text_field=self.config["train_cln_name"],
                                    args=self.set_training_args(),
                                    tokenizer=self.tokenizer,
                                    packing=self.config["packing"],
                                    max_seq_length=self.config["max_seq_length"]
                                )

    def train_and_save_model(self):
        self.create_trainer()
        self.trainer.train()
        self.trainer.save_model(self.config["output_dir"])
        self.tokenizer.save_pretrained(self.config["output_dir"])

In [ ]:
def create_data():
    data = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset", split="train")
    data_df = data.to_pandas()
    data_df = data_df[:5000]
    data_df["text"] = data_df[["instruction", "response"]].apply(lambda x: "Human: " + x["instruction"] + " " + " Assistant: "+ x["response"], axis=1)
    data = Dataset.from_pandas(data_df)
    return data

data = create_data()

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
train_sft = TrainSFT(data, sft_config)
train_sft.train_and_save_model()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

trainable params: 1,843,200 || all params: 2,508,015,616 || trainable%: 0.073492365368111
None


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (defa

Step,Training Loss
100,1.947600
200,1.368500


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


# Inference

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("Vasanth/gemma-chatbot")

inputs = tokenizer("""
###Human: I have a problem in my order 123. So I want to cancel it
###Assistant: """, return_tensors="pt").to("cuda")

model = AutoPeftModelForCausalLM.from_pretrained(
    "Vasanth/gemma-chatbot",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda",
    load_in_4bit=True)

generation_config = GenerationConfig(
    max_new_tokens=100,
    pad_token_id=tokenizer.eos_token_id
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')



###Human: I have a problem in my order 123. So I want to cancel it
###Assistant:  "{{Assistant Name}}  I understand that you have a problem with your order number {{Order Number}}. I would be happy to assist you with the cancellation process. To cancel your order, please provide me with the necessary details such as the order number, any additional information, or any other relevant information. I will do my best to provide you with the best possible assistance. If you have any further questions or concerns, please do not hesitate to reach out to us. Thank you for your understanding and cooperation
14.218164920806885
